In [1]:
import pandas as pd
import wrangle as w
import explore as e
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from functools import reduce

import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

In [2]:
# acquire dataframe
df = w.get_perpetrator_df()

# create production dataframe based on the target variables
production_df = df[df['gname'].str.contains('Unknown')==True]

# create target variable dataframe
df1 = df[df['gname'].str.contains('Unknown')==False]

# drop columns from the dataframe that arent relevant in the predictor
data = df1.drop(columns=['event_id',
             'month',
             'day',
             'region',
             'latitude',
             'longitude',
             'success',
             'attacktype_n',
             'targtype_n',
             'targsubtype_n',
             'targsubtype',
             'corp1',
             'target_description',
             'nationality',
             'claimed',
             'weapdesc',
             'nkillus',
             'nkillter',
             'nwound',
             'nwoundus',
             'nwoundte',
             'property'])
data

,year,country,provstate,city,suicide,attack_type,targtype,gname,weaptype,nkill
date,,,,,,,,,,
2001-04-04,2001,iraq,Dihok,Duhok,0,Bombing/Explosion,Government (General),Kurdistan Workers' Party (PKK),Explosives,7.0
2001-09-10,2001,afghanistan,Takhar,Khodja Baha ud din,1,Bombing/Explosion,Government (General),Taliban,Explosives,3.0
2001-09-26,2001,afghanistan,Kabul,Kabul,0,Facility/Infrastructure Attack,Government (Diplomatic),Taliban,Incendiary,0.0
2002-05-02,2002,afghanistan,Khost,Khost,0,Bombing/Explosion,Military,Taliban,Explosives,0.0
2002-11-21,2002,afghanistan,Kabul,Kabul,0,Bombing/Explosion,Private Citizens & Property,Taliban,Explosives,0.0
...,...,...,...,...,...,...,...,...,...,...
2017-12-31,2017,afghanistan,Logar,Mohammad Agha district,0,Bombing/Explosion,Police,Taliban,Explosives,5.0
2017-12-31,2017,iraq,Diyala,Muqdadiyah,0,Armed Assault,Military,Islamic State of Iraq and the Levant (ISIL),Firearms,13.0
2017-12-31,2017,somalia,Bakool,Wajid,0,Bombing/Explosion,Military,Al-Shabaab,Explosives,0.0


### Clustering

In [3]:
# importing necessary libraries
import pandas as pd
import numpy as np
# !pip install kmodes
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
cluster_data = data
cluster_data = cluster_data.drop(columns={'gname'})
cluster_data

,Cluster,year,country,provstate,city,suicide,attack_type,targtype,weaptype,nkill
date,,,,,,,,,,
2001-04-04,0,2001,iraq,Dihok,Duhok,0,Bombing/Explosion,Government (General),Explosives,7.0
2001-09-10,1,2001,afghanistan,Takhar,Khodja Baha ud din,1,Bombing/Explosion,Government (General),Explosives,3.0
2001-09-26,1,2001,afghanistan,Kabul,Kabul,0,Facility/Infrastructure Attack,Government (Diplomatic),Incendiary,0.0
2002-05-02,0,2002,afghanistan,Khost,Khost,0,Bombing/Explosion,Military,Explosives,0.0
2002-11-21,0,2002,afghanistan,Kabul,Kabul,0,Bombing/Explosion,Private Citizens & Property,Explosives,0.0
...,...,...,...,...,...,...,...,...,...,...
2017-12-31,0,2017,afghanistan,Logar,Mohammad Agha district,0,Bombing/Explosion,Police,Explosives,5.0
2017-12-31,2,2017,iraq,Diyala,Muqdadiyah,0,Armed Assault,Military,Firearms,13.0
2017-12-31,0,2017,somalia,Bakool,Wajid,0,Bombing/Explosion,Military,Explosives,0.0


# Elbow curve to find optimal K
cost = []
K = range(1,5)
for num_clusters in list(K):
    kmode = KModes(n_clusters=num_clusters, init = "random", n_init = 5, verbose=1)
    kmode.fit_predict(data)
    cost.append(kmode.cost_)
    
plt.plot(K, cost, 'bx-')
plt.xlabel('No. of clusters')
plt.ylabel('Cost')
plt.title('Elbow Method For Optimal k')
plt.show()

In [5]:
# Building the model with 3 clusters
kmode = KModes(n_clusters=3, init = "random", n_init = 5, verbose=1)
clusters = kmode.fit_predict(cluster_data)
clusters

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 3451, cost: 62443.0
Run 1, iteration: 2/100, moves: 2078, cost: 62093.0
Run 1, iteration: 3/100, moves: 491, cost: 62093.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 2890, cost: 63871.0
Run 2, iteration: 2/100, moves: 15, cost: 63871.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 3553, cost: 63163.0
Run 3, iteration: 2/100, moves: 534, cost: 63163.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 4437, cost: 61747.0
Run 4, iteration: 2/100, moves: 593, cost: 61747.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 2062, cost: 63892.0
Best run was number 4


array([0, 1, 1, ..., 0, 2, 2], dtype=uint16)

In [6]:
data.insert(0, "Cluster", clusters, True)
data

,Cluster,year,country,provstate,city,suicide,attack_type,targtype,gname,weaptype,nkill
date,,,,,,,,,,,
2001-04-04,0,2001,iraq,Dihok,Duhok,0,Bombing/Explosion,Government (General),Kurdistan Workers' Party (PKK),Explosives,7.0
2001-09-10,1,2001,afghanistan,Takhar,Khodja Baha ud din,1,Bombing/Explosion,Government (General),Taliban,Explosives,3.0
2001-09-26,1,2001,afghanistan,Kabul,Kabul,0,Facility/Infrastructure Attack,Government (Diplomatic),Taliban,Incendiary,0.0
2002-05-02,0,2002,afghanistan,Khost,Khost,0,Bombing/Explosion,Military,Taliban,Explosives,0.0
2002-11-21,0,2002,afghanistan,Kabul,Kabul,0,Bombing/Explosion,Private Citizens & Property,Taliban,Explosives,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31,0,2017,afghanistan,Logar,Mohammad Agha district,0,Bombing/Explosion,Police,Taliban,Explosives,5.0
2017-12-31,2,2017,iraq,Diyala,Muqdadiyah,0,Armed Assault,Military,Islamic State of Iraq and the Levant (ISIL),Firearms,13.0
2017-12-31,0,2017,somalia,Bakool,Wajid,0,Bombing/Explosion,Military,Al-Shabaab,Explosives,0.0


#### Rerunning DTC

In [27]:
# create a dataframe for the gname target variable
gnames = pd.DataFrame(data.gname)

# create nkill df
nkills = pd.DataFrame(data.nkill)

# scale nkill
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(nkills)
nkills = pd.DataFrame(scaler.transform(nkills))

In [31]:
gnames.value_counts()

gname                                      
Taliban                                        4806
Islamic State of Iraq and the Levant (ISIL)    3700
Al-Shabaab                                     1558
Kurdistan Workers' Party (PKK)                  949
Tehrik-i-Taliban Pakistan (TTP)                 870
Al-Qaida in Iraq                                495
Sinai Province of the Islamic State             367
Khorasan Chapter of the Islamic State           271
Baloch Republican Army (BRA)                    265
dtype: int64

In [8]:
# rename popped off index
nkills = nkills.rename(columns={0:'nkill'})
nkills

,nkill
0,0.010448
1,0.004478
2,0.000000
3,0.000000
4,0.000000
...,...
13276,0.007463
13277,0.019403
13278,0.000000
13279,0.008955


In [9]:
data.shape[0]

13281

In [10]:
gnames.shape[0]

13281

In [11]:
nkills.shape[0]

13281

In [12]:
data.year = data.year.astype('str')

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13281 entries, 2001-04-04 to 2017-12-31
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Cluster      13281 non-null  uint16 
 1   year         13281 non-null  object 
 2   country      13281 non-null  object 
 3   provstate    13281 non-null  object 
 4   city         13281 non-null  object 
 5   suicide      13281 non-null  int64  
 6   attack_type  13281 non-null  object 
 7   targtype     13281 non-null  object 
 8   gname        13281 non-null  object 
 9   weaptype     13281 non-null  object 
 10  nkill        13281 non-null  float64
dtypes: float64(1), int64(1), object(8), uint16(1)
memory usage: 1.1+ MB


In [14]:
# encode variables
dummy_df = pd.get_dummies(data[['country',
                                    'provstate',
                                    'city',
                                    'suicide',
                                    'attack_type', 
                                    'targtype',  
                                    'weaptype',
                                    'year', 
                                    'Cluster']], dummy_na=False, drop_first=[True, True])


In [15]:
dummy_df = dummy_df.reset_index(drop=True)
dummy_df.index

RangeIndex(start=0, stop=13281, step=1)

In [16]:
gnames = gnames.reset_index(drop=True)
gnames.index

RangeIndex(start=0, stop=13281, step=1)

In [17]:
# add target to dataframe
dummy_df = pd.concat([dummy_df, gnames, nkills], axis=1)

In [18]:
dummy_df.shape

(13281, 3452)

In [19]:
# drop previous index, date column
dummy_df.head()

suicide  Cluster  country_bahrain  country_djibouti  country_egypt  \
0        0        0                0                 0              0   
1        1        1                0                 0              0   
2        0        1                0                 0              0   
3        0        0                0                 0              0   
4        0        0                0                 0              0   

   country_iran  country_iraq  country_israel  country_jordan  \
0             0             1               0               0   
1             0             0               0               0   
2             0             0               0               0   
3             0             0               0               0   
4             0             0               0               0   

   country_lebanon  country_pakistan  country_saudi arabia  country_somalia  \
0                0                 0                     0                0   
1                0                 0                     0                0   
2                0                 0                     0                0   
3                0                 0                     0                0   
4                0                 0                     0                0   

   country_syria  country_turkey  provstate_Adiyaman  provstate_Agri  \
0              0               0                   0               0   
1              0               0                   0               0   
2              0               0                   0               0   
3              0               0                   0               0   
4              0               0                   0               0   

   provstate_Al Anbar  provstate_Al Hasakah  provstate_Al Qadisiyah  \
0                   0                     0                       0   
1                   0                     0                       0   
2                   0                     0                       0   
3                   0                     0                       0   
4                   0                     0                       0   

   provstate_Al Sharqia  provstate_Aleppo  provstate_Amman  provstate_Ankara  \
0                     0                 0                0                 0   
1                     0                 0                0                 0   
2                     0                 0                0                 0   
3                     0                 0                0                 0   
4                     0                 0                0                 0   

   provstate_Antalya  provstate_Arbil  provstate_Ardahan  provstate_Artvin  \
0                  0                0                  0                 0   
1                  0                0                  0                 0   
2                  0                0                  0                 0   
3                  0                0                  0                 0   
4                  0                0                  0                 0   

   provstate_As Suwayda  provstate_Asir  provstate_At Tamim  \
0                     0               0                   0   
1                     0               0                   0   
2                     0               0                   0   
3                     0               0                   0   
4                     0               0                   0   

   provstate_Azad Kashmir  provstate_Babil  provstate_Badakhshan  \
0                       0                0                     0   
1                       0                0                     0   
2                       0                0                     0   
3                       0                0                     0   
4                       0                0                     0   

   provstate_Badghis  provstate_Baghdad  provstate_Baghlan  provstate_Bakool  \
0                  0

In [20]:
def train_validate_test_split(df, target, seed=123):
    '''
    This function takes in a dataframe, the name of the target variable
    (for stratification purposes), and an integer for a setting a seed
    and splits the data into train, validate and test. 
    Test is 20% of the original dataset, validate is .30*.80= 24% of the 
    original dataset, and train is .70*.80= 56% of the original dataset. 
    The function returns, in this order, train, validate and test dataframes. 
    '''
    train_validate, test = train_test_split(df, test_size=0.27, 
                                            random_state=seed, 
                                            stratify=df[target])
    train, validate = train_test_split(train_validate, test_size=0.35, 
                                       random_state=seed,
                                       stratify=train_validate[target])
    return train, validate, test

In [21]:
# split into train, validate, test
train, validate, test = train_validate_test_split(dummy_df, target='gname', seed=123)

# create X & y version of train, where y is a series with just the target variable and X are all the features. 

X_train = train.drop(columns=['gname'])
y_train = train.gname

X_validate = validate.drop(columns=['gname'])
y_validate = validate.gname

X_test = test.drop(columns=['gname'])
y_test = test.gname

In [37]:
metrics = []
for i in range(1, 15):
    tree = DecisionTreeClassifier(max_depth=i, random_state=123)
    tree = tree.fit(X_train, y_train)
    in_sample_accuracy = tree.score(X_train, y_train)
    
    out_of_sample_accuracy = tree.score(X_validate, y_validate)
    output = {
        "max_depth": i,
        "train_accuracy": in_sample_accuracy,
        "validate_accuracy": out_of_sample_accuracy
    }
    
    metrics.append(output)
    
dt_df = pd.DataFrame(metrics)
dt_df["difference"] = dt_df.train_accuracy - dt_df.validate_accuracy
dt_df

,max_depth,train_accuracy,validate_accuracy,difference
0,1,0.607205,0.606364,0.000841
1,2,0.745437,0.746022,-0.000585
2,3,0.820187,0.819976,0.000211
3,4,0.887002,0.886859,0.000143
4,5,0.931598,0.930760,0.000838
5,6,0.959372,0.957278,0.002094
6,7,0.965402,0.960224,0.005178
7,8,0.967942,0.959045,0.008896
8,9,0.973179,0.960224,0.012955
9,10,0.976353,0.963170,0.013183


In [38]:
X_train.shape, X_validate.shape, X_test.shape

((6301, 3451), (3394, 3451), (3586, 3451))

In [40]:
tree = DecisionTreeClassifier(max_depth=4, random_state=123)

tree = tree.fit(X_train, y_train)

tree.score(X_test, y_test)

0.8881762409369771